# Tutorial 2.0.1: Imports

Below are all the python imports required to run Tutorial 2. This notebook will be called from each other notebook so that the imports will be available in all the notebooks.

This adds MB-Fit to your path so that it can be imported below. If you are running this notebook at QDMS 2025, the below path will work for you, if not, you will have to specify the path to your MB-Fit installation or make sure MB-Fit is on your PATH or PYTHONPATH another way.

In [1]:
import sys
sys.path.append("/expanse/projects/qstore/csd973/MB-Fit_dev")

This is a bit of Jupyter magic that is mostly useful during development and testing.

In [2]:
%load_ext autoreload
%autoreload 2

This tells matplotlib how we want to render our figures and animations.

In [3]:
%matplotlib agg

Common python libraries:

In [4]:
# import numpy
from IPython.display import HTML
import itertools

`mbfit.system` contains the backbone of MB-Fit, the `System` and associated classes:

In [5]:
from mbfit.system import BasicCompound
from mbfit.system.system_like import BondType
from mbfit.system.definition import BasicFragmentDefinition, BasicCompoundDefinition, AtomDefinition, BondDefinition
from mbfit.system.initializer import VSEPRInitializer
from mbfit.system.descriptor import fragment_definition_to_xyz, compound_definition_to_xyz, compound_to_xyz, compound_to_commented_xyz
from mbfit.system.parser import xyz_to_compound_definition, xyz_to_compound_definition, xyz_to_compound_by_definition
from mbfit.system.transformer import SystemTransformer
from mbfit.system.copier import CompoundCopier
from mbfit.system.transformer import make_scan
from mbfit.system.region import BasicMutableRegionManager
# from mbfit.system.descriptor.energy_decomposition_comment_line_descriptor import EnergyDecompositionCommentLineDescriptor

from mbfit.system.io import write_fragment_definition, write_definition, write_system, write_systems, read_fragment_definition, read_definition, read_system, read_systems, write_system_and_energy, write_systems_and_energies, read_system_and_energy, read_systems_and_energies
from mbfit.system.io import write_system_and_nmer_energies, write_systems_and_nmer_energies, read_system_and_nmer_energy, read_systems_and_nmer_energies
from mbfit.system.io import write_training_set_file

`mbfit.calculator` contains wrappers for QM software and functions to help us manipulate electronic structure data:

In [6]:
from mbfit.calculator import QchemCalculator, OrcaCalculator, Psi4Calculator
from mbfit.calculator import calculate_energy, calculate_energies, minimize_structures, minimize_structure, calculate_vibrational_modes, write_vibrational_modes, read_vibrational_modes, calculate_nmer_energies, find_clusters
from mbfit.calculator import calculate_many_body_energies, calculate_many_body_energy, calculate_binding_energies, calculate_binding_energy
from mbfit.calculator import write_multiple_vibrational_modes, read_multiple_vibrational_modes, discard_failed_configurations

`mbfit.visualization` contains functions to make plots and structure renders:

In [7]:
from mbfit.visualization import render_system, render_systems, render_overlayed_systems
from mbfit.visualization import plot_scan, plot_clusters, plot_correlation

`mbfit.parametrization` will allow us to parametrize $V_{phys}$ and the PIPs:

In [8]:
from mbfit.polynomials_old import generate_polynomial
from mbfit.parametrization import calculate_polarizabilities_by_XDM, calculate_radii_by_XDM, calculate_C6_by_XDM, calculate_d6_from_radii, get_ESP_grid_points, calculate_ESP_at_grid_points, fit_charges_to_ESP
from mbfit.parametrization import generate_pip_parametizer, parametrize_pip, evaluate_pip, read_parameters, write_parameters

`mbfit.configurations` has tools to generate configurations for our training sets:

In [9]:
from mbfit.configurations import NormalModesConfigurationGenerator, get_pbmetad_parameters, perform_pbmetad_simulation, RandomDistanceAndRotationConfigurationGenerator

`mbfit.potential` has the functions to evaluate $V_{phys}$ and other energy contributions.

In [10]:
from mbfit.potential import calculate_Vphys_energies, calculate_Vphys_nmer_energies

`mbfit.sampling` has the tools we will use to choose the configurations for our training set.

In [11]:
from mbfit.sampling import perform_fps, split_pool, SplitStrategy, filter_configs_by_cutoff

`mbfit.implementation` will help us generate the C++ code required to evaluate the PEF in MBX.

In [12]:
from mbfit.implementation import initialize_mbx, add_monomer_type_to_MBX, add_excluded_pairs, add_charges_to_MBX, add_polarizability_factors_to_MBX, add_polarizabilities_to_MBX, add_reciprocal_space_C6_to_MBX, add_C6_and_d6_to_MBX, compile_mbx, add_PIP_to_MBX, calculate_MBX_energy, calculate_MBX_nmer_energies

Finally, `mbfit.utils` contains a few helpful functions.

In [13]:
from mbfit.utils.filesystem import Directory
# from mbfit.utils.job_manager import SLURMJobManager, JobManager, LocalJobManager
# from mbfit.utils import constants

## Don't look here (this part is intended to be removed before the workshop, so if you see this I forgot.)

In [17]:
# qchem_job_template="""
# module load cpu/0.15.4  gcc/10.2.0  mvapich2/2.3.6 qchem/5.4
# conda activate p310
# cd ~/software/development/MB-Fit_versions/QDMS_2025
# export MBFIT_HOME=$PWD
# source sourceme.sh
# cd -

# export QCSCRATCH=/expanse/lustre/scratch/$USER/temp_project/qscratch
# export QCLOCALSCR=/scratch/$USER/job_$SLURM_JOB_ID

# """

# def get_qchem_job_manager(job_scratch_directory: Directory) -> JobManager:
#     job_scratch_directory.create()
#     return LocalJobManager()
#     return SLURMJobManager(
#             input_path=f"{job_scratch_directory}/job.in",
#             output_path=f"{job_scratch_directory}/job.out",
#             log_path=f"{job_scratch_directory}/job.log",
#             job_path=f"{job_scratch_directory}/job.job",
#             job_output_path=f"{job_scratch_directory}/job.stdout",
#             job_error_path=f"{job_scratch_directory}/job.stderr",
#             max_queue_size=10,
#             job_name="slurm_job_manager",
#             allocation="csd888",
#             reservation=None,
#             partition="shared",
#             qos=None,
#             max_walltime_minutes=60,
#             nodes=1,
#             tasks=1,
#             processors_per_task=32,
#             template=qchem_job_template
#     )

In [1]:
psi4_job_template="""
source ~/.bashrc
conda activate qdms_tutorial_2

cd /expanse/projects/qstore/csd973/MB-Fit_dev
export MBFIT_HOME=$PWD
source sourceme.sh
cd -
"""

from mbfit.utils.job_manager import SLURMJobManager, CompoundJobManager

def get_psi4_job_manager(job_scratch_directory: Directory):
    job_scratch_directory.create()
    return CompoundJobManager(
        [
            SLURMJobManager(
                input_path=f"{job_scratch_directory}/job{i}.in",
                output_path=f"{job_scratch_directory}/job{i}.out",
                log_path=f"{job_scratch_directory}/job{i}.log",
                job_path=f"{job_scratch_directory}/job{i}.job",
                job_output_path=f"{job_scratch_directory}/job{i}.stdout",
                job_error_path=f"{job_scratch_directory}/job{i}.stderr",
                max_queue_size=10,
                job_name="slurm_job_manager{i}",
                allocation="csd973",
                reservation=None,
                partition="shared",
                qos=None,
                max_walltime_minutes=120,
                nodes=1,
                tasks=1,
                processors_per_task=4,
                template=psi4_job_template
            )
            for i in range(10)
        ]
    )
    return SLURMJobManager(
            input_path=f"{job_scratch_directory}/job.in",
            output_path=f"{job_scratch_directory}/job.out",
            log_path=f"{job_scratch_directory}/job.log",
            job_path=f"{job_scratch_directory}/job.job",
            job_output_path=f"{job_scratch_directory}/job.stdout",
            job_error_path=f"{job_scratch_directory}/job.stderr",
            max_queue_size=10,
            job_name="slurm_job_manager",
            allocation="csd973",
            reservation=None,
            partition="shared",
            qos=None,
            max_walltime_minutes=60,
            nodes=1,
            tasks=1,
            processors_per_task=4,
            template=psi4_job_template
    )

ModuleNotFoundError: No module named 'mbfit'